In [79]:
from dataset_builder import EEDatasetBuilder
import geemap
import ee
import os
import numpy as np

In [38]:
Map = geemap.Map(center=[29.7666636 , 78.1999992], zoom=9)
Map

Map(center=[29.7666636, 78.1999992], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…

In [3]:
# Build ee dataset builder
ee_dataset_builder = EEDatasetBuilder()

In [4]:
# Ganga river buffered zone, used local shapefile

ganga_buffer_shp = r'C:\Users\warne\Desktop\Projects\naalas_2\Data\Naalas&Ganga_Buff\buff_shp.shp'
ganga_buffer = geemap.shp_to_ee(ganga_buffer_shp)
#Map.addLayer(ganga_buffer, {}, 'Ganga')

In [17]:
# Not actually an image, it is a table/feature collection, is used to create buffer zone for other files called, 
# Need an image accessable on google earth engine
ganga_buff = ee.FeatureCollection('projects/nikhilrajdeep/assets/Buffer_Ganga_river')

In [25]:
#Doesn't work long term but current place holder to get the ee_dataset_builder file to work
ganga_cheat = ee.Image('projects/nikhilrajdeep/assets/Buffer_Ganga_river')

In [6]:
# My attempt at getting an image up from Nikhils work, doesnt work
# ganga_2 = ee.Image('projects/ee-warnermichael09/assets/Ganga_buffer')
# Map.addLayer(ganga_2, {}, 'Ganga_2')

In [26]:
# Need to convert the buffer zone to an image, currently using a feature collection
ee_dataset_builder.filtered_response_layer_from_raster(
    response_raster='custom', 
    ee_image=ganga_cheat,
    custom_response_raster_name='response'
)

PD
LULC
Prec
DD
Slope
LST #land surface temp

# Show where to get the geoTIFF files from

In [27]:
out_dir = os.path.expanduser(r'\Data\geotiffs')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

PD = os.path.join(r"Data\geotiffs\pop_bucket.tif")
LULC = os.path.join(r"C:\Users\warne\Desktop\Projects\naalas_2\Data\geotiffs\lulcCond.tif")
Prec = os.path.join(r'C:\Users\warne\Desktop\Projects\naalas_2\Data\geotiffs\rainCond.tif')
DD = os.path.join(r"Data\geotiffs\ddCond.tif")
Slope = os.path.join(r'Data\geotiffs\slopeCond.tif')
LST = os.path.join(r'Data\geotiffs\tempCond.tif')



In [28]:
#Population Density
Map.add_raster(PD, layer_name = "Population Density")
Map.add_raster(LULC, layer_name = "Land use, Land cover")
Map.add_raster(Prec, layer_name = "Rainfall")
Map.add_raster(DD, layer_name = "Drainage Density")
Map.add_raster(Slope, layer_name = "Slope")
Map.add_raster(LST, layer_name = "Land Surface Temp")

### Chirps is the rain fall dataset

In [29]:
merit_water = ee.Image('MERIT/Hydro/v1_0_1');
ele = ee.Image("NASA/NASADEM_HGT/001");
twi = ee.Image("projects/nikhilrajdeep/assets/TWI_Ganga");
dd = ee.Image("projects/nikhilrajdeep/assets/drainage_density");
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD");
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2");
lc = ee.ImageCollection("ESA/WorldCover/v200");
soil = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02");
pop = ee.ImageCollection("CIESIN/GPWv411/GPW_UNWPP-Adjusted_Population_Density");

# Population Density

In [39]:

gpop = pop.toBands().select('gpw_v4_population_density_adjusted_to_2015_unwpp_country_totals_rev11_2020_30_sec_unwpp-adjusted_population_density').clip(ganga_buff).rename('popDensity');

In [63]:
gpop_cond = gpop.expression(
    "(b('popDensity') < 865) ? 1" +
    ":(b('popDensity') >= 865 && b('popDensity') < 1600) ? 2" +
    ":(b('popDensity') >= 1600 && b('popDensity') < 2680) ? 3" +
    ":(b('popDensity') >= 2680 && b('popDensity') < 4036) ? 4" +
    ":(b('popDensity') >= 4036) ? 5" +
    ": 0"  # Default value in case none of the conditions above are met
)

In [68]:
conditionParams = {
    'min': 1,
    'max': 5,
    'palette': ['1a9641', 'a6d96a', 'ffffbf', 'fdae61', 'd7191c']
}

In [69]:
Map.addLayer(gpop_cond, conditionParams, 'Population density')

In [116]:
predictors = gpop_cond.rename('PD')

# Land Use Land Cover, LULC

In [111]:
lulc = lc.first().clip(ganga_buff).rename('LULC')

In [112]:
lulcCond = (lulc.eq(80).multiply(5)
            .where(lulc.eq(90), 1)
            .where(lulc.eq(10), 1)
            .where(lulc.eq(20), 1)
            .where(lulc.eq(30), 1)
            .where(lulc.eq(40), 4)
            .where(lulc.eq(60), 2)
            .where(lulc.eq(50), 3))

In [113]:
Map.addLayer(lulcCond, conditionParams, 'LULC');

In [117]:
predictors = predictors.addBands(lulcCond)

# Precipitation, chirps dataset




In [71]:
rain = chirps.filterDate('2022-01-01','2023-01-01').sum().clip(ganga_buff)

In [91]:
rainCond = (rain
  .where(rain.lt(868), 1)
  .where(rain.gte(868)and(rain.lt(1019)), 2)
  .where(rain.gte(1019)and(rain.lt(1189)), 3)
  .where(rain.gte(1189)and(rain.lt(1410)), 4)
  .where(rain.gte(1410), 5))


In [92]:
Map.addLayer(rainCond, conditionParams, 'Rainfall');

In [118]:
predictors = predictors.addBands(rainCond)

# Drainage Density DD

Drainage density

| DD         | Condition     | 

| <0.53      | Extreme low   |
 
 | 0.53-1.43  | low           |
 
 | 1.43-2.57  | Moderate      |
 
 | 2.57-4     | high          |
 
 | >4         | Extreme high  |

In [100]:
ddCond = (dd
  .where(dd.lt(0.6), 1)
  .where(dd.gte(0.6)and(dd.lt(1.5)), 2)
  .where(dd.gte(1.5)and(dd.lt(2.6)), 3)
  .where(dd.gte(2.6)and(dd.lt(4)), 4)
  .where(dd.gte(4), 5)).rename('drainage_density')

In [97]:
Map.addLayer(ddCond, conditionParams, 'drainage density');

In [119]:
predictors = predictors.addBands(ddCond)

# Slope

In [121]:
dem = ele.select('elevation')
slope = ee.Terrain.slope(dem).clip(ganga_buff)

In [122]:
slopeCond = (slope
  .where(slope.lt(4), 5)
  .where(slope.gte(4)and(slope.lt(12)), 4)
  .where(slope.gte(12)and(slope.lt(24)), 3)
  .where(slope.gte(24)and(slope.lt(35)), 2)
  .where(slope.gte(35), 1))

In [123]:
Map.addLayer(slopeCond, conditionParams, 'slope');

In [124]:
predictors = predictors.addBands(slopeCond)

# Land Surface Temperature LST

# Merit - Hydro

In [14]:
twi_cond = (twi.where(twi.lt(6.4), 1)
                .where(twi.gte(6.4).And(twi.lt(8.3)), 2)
                .where(twi.gte(8.3).And(twi.lt(10.3)), 3)
                .where(twi.gte(10.3).And(twi.lt(13.5)), 4)
                .where(twi.gte(13.5), 5))

# Define visualization parameters in a dictionary.
condition_params = {
    'min': 1,
    'max': 5,
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']  # Example palette
}

# Use Map to add layers in the Earth Engine Python API.
Map = geemap.Map()  # geemap might be used to display Earth Engine data layers
Map.addLayer(twi_cond, condition_params, 'TWI')

In [113]:
predictors = merit_water

In [115]:
predictors = predictors.addBands(twi_cond)

In [116]:
Map.addLayer(merit_water)

In [40]:
## WETLANDS ##

# Elevation - MERIT DEM
# Elevation and slope, which can be derived, should be core variables in determining where CFWs form.
dem = ee.Image('MERIT/DEM/v1_0_3');
predictors = dem.rename('elevation')

# Calculate slope in degrees
slope = ee.Terrain.slope(dem).rename('slope'); # fix this ? why is it not in degrees?
predictors = predictors.addBands(slope)

# Hydrology - MERIT HYDRO 
# Distance to water features can be determined with this data. CFW are likely to be close to water features.
merit_water = ee.Image('MERIT/Hydro/v1_0_1');
# Compute distance to water
water_mask = merit_water.select("viswth");
distance_to_water = water_mask.distance(ee.Kernel.euclidean(1000, 'meters')).rename('distance_water')
predictors = predictors.addBands(distance_to_water)

# JRC Global Surface Water
# Distance measures can be leveraged to understand proximity to water.
global_surface_water = ee.Image('JRC/GSW1_4/GlobalSurfaceWater');
# Compute distance to water
water_mask_global = global_surface_water.select("occurrence");
mask = water_mask_global.neq(0).rename('mask');
# Apply the mask to the image.
masked = water_mask_global.updateMask(mask);
# Set all masked values to 1.
water_mask_global_binary = masked.where(mask, 1).rename('global_water').unmask()
proximity_to_water = water_mask_global_binary.distance(ee.Kernel.euclidean(7000, 'meters'))
proximity_to_water = proximity_to_water.unmask().rename('proximity_water')
predictors = predictors.addBands(proximity_to_water)

# Soil moisture - Enhanced SMAP
# Wetlands tend to have much high soil moisture content compared to surrounding habitats.
dataset = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture').filter(ee.Filter.date('2016-01-01', '2016-12-30'));
soilMoisture = dataset.select('ssm').mean().rename('soil_moisture_mean');
predictors = predictors.addBands(soilMoisture) 

# # climate - Daymet
# # The various parameters can be summarized to determine the climate patterns.
# daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4').filter(ee.Filter.date('2016-01-01', '2016-12-30'));
# maximumTemperature = daymet.select('tmax').mean().rename('tmax_mean');
# minTemperature = daymet.select('tmin').mean().rename('tmin_mean');
# precipitation = daymet.select('prcp').mean().rename('precipitation_mean');
# predictors = predictors.addBands(maximumTemperature) 
# predictors = predictors.addBands(minTemperature) 
# predictors = predictors.addBands(precipitation) 

# Climate: NEX-GDDP-CMIP6: NASA Earth Exchange Global Daily Downscaled Climate Projections
nex_gppd_cmip6 = ee.ImageCollection("NASA/GDDP-CMIP6").filter(ee.Filter.date('2016-01-01', '2016-12-30')).filter(ee.Filter.eq('model', 'ACCESS-CM2'));
maximumTemperature = nex_gppd_cmip6.select('tasmax').mean().rename('tmax_mean');
minTemperature = nex_gppd_cmip6.select('tasmin').mean().rename('tmin_mean');
precipitation = nex_gppd_cmip6.select('pr').mean().rename('precipitation_mean');
predictors = predictors.addBands(maximumTemperature) 
predictors = predictors.addBands(minTemperature) 
predictors = predictors.addBands(precipitation) 


# Litology, Landforms, Physiography – Physical descriptions of the landscape, which can be used to understand landscape context.
# Soil lithology
lithology = ee.Image('CSP/ERGo/1_0/US/lithology').rename('lithology');
predictors = predictors.addBands(lithology)

# Topography: Landforms
landforms = ee.Image('CSP/ERGo/1_0/US/landforms');
landforms = landforms.select('constant').rename('landforms');
predictors = predictors.addBands(landforms)

# Soil and topography
physiography = ee.Image('CSP/ERGo/1_0/US/physiography');
physiography = physiography.select('constant').rename('physiography');
predictors = predictors.addBands(physiography)

In [41]:
predictors.bandNames().getInfo()

['elevation',
 'slope',
 'distance_water',
 'proximity_water',
 'soil_moisture_mean',
 'tmax_mean',
 'tmin_mean',
 'precipitation_mean',
 'lithology',
 'landforms',
 'physiography']

In [42]:
vis_param = { 
  'min': 0.0,
  'max': 1.0,
  'palette':  [
    'white', 'blue'
  ]}

Map.addLayer(ee_dataset_builder.image.select('response'), vis_param, 'CCAP 13 + 16 bands masked mangroves', True)

EEException: Image.load: Asset 'projects/nikhilrajdeep/assets/Buffer_Ganga_river' is not an Image.

In [43]:
elevation_vis_param = { 
  'min': 240,
  'max': 310,
  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
    'format': 'png'};

Map.addLayer(ee_dataset_builder.image.select('tmin_mean'), elevation_vis_param, 'minTemperature', True)

EEException: Image.load: Asset 'projects/nikhilrajdeep/assets/Buffer_Ganga_river' is not an Image.

In [ ]:
elevation_vis_param = { 
  'min': -3,
  'max': 18.0,
  'palette':  ['000000', '478FCD', '86C58E', 'AFC35E', '8F7131',
          'B78D4F', 'E2B8A6', 'FFFFFF'],
    'format': 'png'};

Map.addLayer(ee_dataset_builder.image.select('elevation'), elevation_vis_param, 'elevation', True)

In [ ]:
Map.addLayer(ee_dataset_builder.image.select('proximity_water'), {'min': 0, 'max': 1000.0, 'palette':  ['white', 'blue']}, 'Proximity to Global Surface Water 100 meters kernel');
